#  導入資料

In [13]:
import pandas as pd 
import numpy as np
robusta=pd.read_csv('robusta_data_cleaned.csv')
arabica=pd.read_csv('arabica_data_cleaned.csv')

#  資料合併

In [14]:
#統一欄名稱
arabica.rename(columns={'Unnamed: 0':'id'},inplace=True)
robusta.rename(columns={'Unnamed: 0':'id','Fragrance...Aroma':'Aroma',\
                     'Salt...Acid':'Acidity','Bitter...Sweet':'Sweetness','Mouthfeel':'Body','Uniform.Cup':'Uniformity'}, inplace=True)
#更正arabica的id
arabica.loc[1310,'id']=1311

#robusta ID接續arabica
robusta.id=robusta.id.map(lambda x:x+1311)

#串接arabica、robusta；修改index
data=pd.concat([arabica,robusta],join='inner')
data.reset_index(inplace=True, drop=True)

#  資料處理

In [15]:
#尋找遺失值欄位
cols_with_missing_value=[col for col in data.columns if data[col].isnull().any()]
#print('cols_with_missing_value:\n',cols_with_missing_value,'\n')

#更正Country.of.Origin名稱、處理缺失值
data['Country.of.Origin'].replace('Cote d?Ivoire','Cote d\'Ivoire',inplace=True)
data['Country.of.Origin'].fillna('Other',inplace=True)
#print('country:\n',data['Country.of.Origin'].unique(),'\n')

#處理Color遺失值、未知值
data.Color.replace('None','Unknown',inplace=True)
data['Color'].fillna('Unknown',inplace=True)

#處理'Owner','Farm.Name','Mill','Company','Region','Producer','Owner.1','Variety','Processing.Method'遺失值
cols_values_unknown=['Owner','Farm.Name','Mill','Company','Region','Producer','Owner.1','Variety','Processing.Method']
data.fillna({col:'Unknown' for col in cols_values_unknown},inplace=True)

#處理'Lot.Number','ICO.Number'遺失值
data.fillna({'Lot.Number':'None','ICO.Number':'None'},inplace=True)


#根據Grading.Date處理Harvest.Year遺失值
data.loc[961,'Harvest.Year']=2017
update_Harvest_Year=data[data['Harvest.Year'].isnull()]['Grading.Date'].map(lambda x: x[-4:])
data.loc[data['Harvest.Year'].isnull(),'Harvest.Year']=update_Harvest_Year
data['Harvest.Year']=data['Harvest.Year'].map(lambda a:str(a)[-4:])
terms=['gust','mmm','TEST','T/10','pril','mber','ulio','bia.','crop']
for i in terms:
    data.loc[data['Harvest.Year']==i,'Harvest.Year']=data[data['Harvest.Year']==i]['Grading.Date'].map(lambda x: x[-4:])

#處理Quakers遺失值
data.loc[366,'Quakers']=0

#處理altitude_mean_meters錯誤值、用同一國家平均值填補遺失值
data.loc[data['altitude_mean_meters']>8000,'altitude_mean_meters']=data[data['altitude_mean_meters']>8000].altitude_mean_meters.map(lambda x:x/100)
data.loc[data['Country.of.Origin']=='Other','altitude_mean_meters']=data.altitude_mean_meters.mean()
country_altitude_mean=data.groupby('Country.of.Origin').altitude_mean_meters.mean().to_dict()
for i in data[data['altitude_mean_meters'].isnull()].index:
    data.loc[i,'altitude_mean_meters']=country_altitude_mean[data.loc[i,'Country.of.Origin']]

#依altitude_mean_meters分類高度
for i in range(len(data)):
    if data.loc[i,'altitude_mean_meters']>3000:
        data.loc[i,'Altitude_classfy']='3000+'
    elif data.loc[i,'altitude_mean_meters']>2000:
        data.loc[i,'Altitude_classfy']='2001-3000'
    elif data.loc[i,'altitude_mean_meters']>1000:
        data.loc[i,'Altitude_classfy']='1001-2000' 
    else:
        data.loc[i,'Altitude_classfy']='0-1000' 

#移除無用欄位       
data.drop(columns=['Altitude', 'altitude_low_meters', 'altitude_high_meters','unit_of_measurement'],inplace=True)

#  輸出CSV檔

In [16]:
#輸出資料
data.to_csv('data.csv',index=False)
print(data.shape)

(1339, 41)
